# VNA V2

Clean up the code to simultaneously read from ADCs. Perform S-parameter caculations and graph output. Completes full data proccessing pipeline and outputs the S-Parameters 

In [19]:
import sys
import os
import importlib #
sys.path.append(os.path.abspath('../scripts'))

#widgets and display
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Output
import pprint

#Plotting 
from bqplot import pyplot as plt
import threading
from threading import Thread

#caculations
import numpy as np 
from scipy import signal
import time
import math

# Import functions from custom scripts
import sig_source
importlib.reload(sig_source)
from sig_source import SigSource


In [20]:
#Import Pynq
from pynq import PL
from pynq import allocate
from pynq import Overlay
import xrfdc

OSError: dlopen(libc.so.6, 0x0006): tried: 'libc.so.6' (no such file), '/usr/lib/libc.so.6' (no such file), '/Users/tklouie/6205_python/ADL5960_RFSoC_VNA/vna_gui/notebooks/libc.so.6' (no such file)

# Upload Code to RFSOC

In [3]:
rfsoc_button = widgets.Button(description="Print RFSOC Code Lines")

out = Output()
def print_rfsoc(func):
    with out:
        out.clear_output
        try: 
            pprint.pprint(ol.ip_dict)
            #nothing! 
        except Exception as e:
            print(f"Error: {e}")


PL.reset() #important fixes caching issues which have popped up.
ol = Overlay('./design_1.bit')  #locate/point to the bit file
dma_interface = ol.axi_dma_0
rf = ol.usp_rf_data_converter_0

rfsoc_button.on_click(print_rfsoc)
display(widgets.VBox([widgets.Label(value="Print RFSOC Output"), rfsoc_button, out])) #Display button only after the code is uploaded 

NameError: name 'PL' is not defined

# Signal Source Generation 

In [4]:
source = SigSource(start = 10000000, stop = 20000000000, resolution = 100)

start_stop_slider = widgets.FloatRangeSlider(
    value=[source.lowest_freq/(10**6), source.highest_freq/(10^6)],
    min= source.lowest_freq/(10**6),
    max= source.highest_freq/(10**6),
    step= 1,
    #description='Start Frequency (MHz)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='1.0f',
    layout=widgets.Layout(width='500px') #Change layout width here!! 
)
center_slider = widgets.FloatSlider(value=source.center_freq/(10**6), min=source.lowest_freq/(10**6), max=source.highest_freq/(10**6), step=1)
span_slider = widgets.FloatSlider(value=((source.highest_freq - source.lowest_freq)/2)/10**6, min=1, max=((source.highest_freq - source.lowest_freq))/10**6, step=1)
resolution_slider = widgets.FloatSlider(value = 100, min = 0, max = 1000, step=1)
calc_selection = widgets.ToggleButtons(
    options=['Start-Stop', 'Center-Span'],
    description='Caculate Based On:',
    disabled=False,
    button_style=''
)
update_button = widgets.Button(description="Update Wave")

out = Output()
def update_function(change):
    with out:
        out.clear_output()
        try:
            if (calc_selection.value == "Start-Stop"):
                source.update_parameters(start = start_stop_slider.value[0]*10**6, 
                                        stop = start_stop_slider.value[1]*10**6, 
                                        resolution = resolution_slider.value)
            elif (calc_selection.value == "Center-Span"):
                source.update_parameters(center = center_slider.value*10**6,
                                        span = span_slider.value*10**6, 
                                        resolution = resolution_slider.value)
            
            start_stop_slider.value = (source.start/10**6, source.stop/10**6)
            center_slider.value = source.center/10**6
            span_slider.value = source.span/10**6
            #print("Start frequency: {} Stop Frequency: {}".format(start_stop_slider.value[0], start_stop_slider.value[1]) )
        except Exception as e:
            print(f"Error: {e}")


b1 = widgets.VBox([widgets.VBox([widgets.Label(value="Start Stop Slider (MHz)"), start_stop_slider]), 
                   widgets.HBox([widgets.VBox([widgets.Label(value="Center Slider (Mhz)"),center_slider]), 
                                 widgets.VBox([widgets.Label("Span Slider (Hz)"), span_slider])]),
                    widgets.VBox([widgets.Label(value="Points per Sweep"), resolution_slider]),
                    calc_selection,
                    update_button,
                    out],
                    layout=widgets.Layout(padding='20px 0'))

update_button.on_click(update_function)
display(b1)

# Real Time Output Plot

In [ ]:
# Sampling frequency
fs = 147.456e6
# Number of samples
n = 65536
T = n/fs

def read_dma():
    # Trigger the DMA transfer and wait for the result
    out_buffer = allocate(400024 * 4, dtype=np.int32)
    # Trigger the DMA transfer and wait for the result
    start_time = time.time()
    dma_interface.recvchannel.transfer(out_buffer)
    dma_interface.recvchannel.wait()
    stop_time = time.time()
    hw_exec_time = stop_time-start_time
    print('Hardware execution time: ',hw_exec_time)
    return out_buffer

def iq_break_data(in_data):
    val = in_data&0xFFFF
    if val >= 32768:
        real = np.int32(0xFFFF0000|val)
    else:
        real = val
    imag = in_data>>16
    return real, imag

# Function to process data
def iq_break_data_np(in_data):
    # Extract real part
    val = in_data & 0xFFFF
    real = np.where(val >= 32768, np.int32(0xFFFF0000 | val), val)
    
    # Extract imaginary part
    imag = in_data >> 16
    return real, imag

'''
Caculates the S-parameters of the two given signals
This caculation must be done to each of the 4 signals to find S11, S12, S21, S22
'''
def caculate_S_param (forward_signal, reverse_signal):
    # Convert signals to polar. This is done by a CORDIC on the FPGA.
    forward_magnitude = np.abs(forward_signal)
    forward_phase = np.angle(forward_signal)
    reverse_magnitude = np.abs(reverse_signal)
    reverse_phase = np.angle(reverse_signal)

    # Reflection coefficient magnitude comes from the ratios of the magnitudes of the forward and reverse signals.
    gamma_magnitude = np.mean(forward_magnitude) / np.mean(reverse_magnitude)

    # Reflection coefficient phase comes from the difference in phase of the forward and reverse signals. 
    # Unwrapping performed by subtracting 2pi when we are outside the 180 deg range. Not 100% sure this works under all conditions.
    phase_difference = forward_phase - reverse_phase
    phase_difference[phase_difference > math.pi] = phase_difference[phase_difference > math.pi] - 2 * math.pi
    gamma_phase = np.mean(phase_difference)

    return gamma_magnitude, gamma_phase

# Real Time Plotting of Signals

In [ ]:
# Plot Size
PLOT_SIZE = 3e-6

# Initialize data lists
time_data = np.linspace(0, T, n)  # Time data (X-axis)
x_axis = [time_data, time_data, time_data, time_data]
# Initialize the plot
fig = plt.figure(title="Real-time Sensor Data", animation_duration=0)
line = plt.plot([], [], colors=["blue", "red", "green", "orange"])  # Initial empty plot

plt.xlim(0, PLOT_SIZE)  # Initial X-axis range, will update dynamically
plt.xlabel("Time [s]")

# Function to update the plot with new sensor data
def update_plot():
    while True:
        if is_running.value:
            out_buffer = read_dma()

            start_time = time.time()
            # Convert the entire out_buffer into a NumPy array
            out_buffer_np = np.array(out_buffer, dtype=np.uint32)  # Assuming `out_buffer` is the raw input array

            # Split into 4 sub-arrays
            out_buffers = np.split(out_buffer_np, 4)

            # Process all buffers at once
            real0, imag0 = iq_break_data_np(out_buffers[0])
            real1, imag1 = iq_break_data_np(out_buffers[1])
            real2, imag2 = iq_break_data_np(out_buffers[2])
            real3, imag3 = iq_break_data_np(out_buffers[3])

            #For caculating the S-parameters combine into complex lists
            array0 = real0 + 1j*imag0 
            array1 = real1 + 1j*imag1
            array2 = real2 + 1j*imag2 
            array3 = real3 + 1j*imag3 
            
            S11_mag, S11_phase = caculate_S_param(array0, array1)
            S12_mag, S12_phase = caculate_S_param(array0, array2)
            S21_mag, S21_phase = caculate_S_param(array1, array3)
            S22_mag, S22_phase = caculate_S_param(array2, array3)

            print(f"Calculated S11 magnitude is {S11_mag:.2f}.")
            print(f"Calculated S11 phase is {S11_phase * 180 / math.pi:.2f}\N{DEGREE SIGN}.")
            print(f"Calculated S12 magnitude is {S12_mag:.2f}.")
            print(f"Calculated S12 phase is {S12_phase * 180 / math.pi:.2f}\N{DEGREE SIGN}.")
            print(f"Calculated S21 magnitude is {S21_mag:.2f}.")
            print(f"Calculated S21 phase is {S21_phase * 180 / math.pi:.2f}\N{DEGREE SIGN}.")
            print(f"Calculated S22 magnitude is {S22_mag:.2f}.")
            print(f"Calculated S22 phase is {S22_phase * 180 / math.pi:.2f}\N{DEGREE SIGN}.")

            stop_time = time.time()
            hw_exec_time = stop_time-start_time
            print('Extract and translate data: ',hw_exec_time)
            
            start_time = time.time()
            # Update the plot with new data
            line.x = x_axis
            line.y = [real0, real1, real2, real3]
            stop_time = time.time()
            hw_exec_time = stop_time-start_time
            print('Plot Time: ',hw_exec_time)
            
# Toggle button to start/stop the plot
is_running = widgets.ToggleButton(
    value=False,
    description="Running",
    icon="play",
    tooltip="Start/Stop the live plot",
)

# Display the toggle button and plot
display(is_running, fig)

# Function to start the thread for continuous plotting
def start_plot(change):
    if is_running.value:
        # Run the update function in a separate thread to avoid blocking the main thread
        thread = Thread(target=update_plot, daemon=True)
        thread.start()

# Watch the button and start the plot when pressed
is_running.observe(start_plot, names='value')

ToggleButton(value=False, description='Running', icon='play', tooltip='Start/Stop the live plot')

Figure(axes=[Axis(label='Time [s]', scale=LinearScale(max=3e-06, min=0.0)), Axis(orientation='vertical', scale…

Exception in thread Thread-4 (update_plot):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/Users/tklouie/6205_python/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/63/5kclp68j4613vhrmzjr_hpk80000gn/T/ipykernel_29875/1803187279.py", line 16, in update_plot
  File "/var/folders/63/5kclp68j4613vhrmzjr_hpk80000gn/T/ipykernel_29875/196679798.py", line 3, in read_dma
NameError: name 'allocate' is not defined


## Now Preparing for Sweeping Output

What needs to happen:
- Loop through setting each of the frequency points on the source
    - In the loop, set the frequency
    - Have some finite measurement in time
    - Read the DMA out for that time
    - Caculate the S-parameters
    - Save that into a plot

- The plot needs to live update linked to those results

- Use: source.generate_freq_points()

In [15]:
def extract_S_param(out_buffer):
    # Convert the entire out_buffer into a NumPy array
    out_buffer_np = np.array(out_buffer, dtype=np.uint32)  # Assuming `out_buffer` is the raw input array

    # Split into 4 sub-arrays
    out_buffers = np.split(out_buffer_np, 4)

    # Process all buffers at once
    real0, imag0 = iq_break_data_np(out_buffers[0])
    real1, imag1 = iq_break_data_np(out_buffers[1])
    real2, imag2 = iq_break_data_np(out_buffers[2])
    real3, imag3 = iq_break_data_np(out_buffers[3])

    #For caculating the S-parameters combine into complex lists
    array0 = real0 + 1j*imag0 
    array1 = real1 + 1j*imag1
    array2 = real2 + 1j*imag2 
    array3 = real3 + 1j*imag3 

    S11_mag, S11_phase = caculate_S_param(array0, array1)
    S12_mag, S12_phase = caculate_S_param(array0, array2)
    S21_mag, S21_phase = caculate_S_param(array1, array3)
    S22_mag, S22_phase = caculate_S_param(array2, array3)
    
    ''' 
    print(f"Calculated S11 magnitude is {S11_mag:.2f}.")
    print(f"Calculated S11 phase is {S11_phase * 180 / math.pi:.2f}\N{DEGREE SIGN}.")
    print(f"Calculated S12 magnitude is {S12_mag:.2f}.")
    print(f"Calculated S12 phase is {S12_phase * 180 / math.pi:.2f}\N{DEGREE SIGN}.")
    print(f"Calculated S21 magnitude is {S21_mag:.2f}.")
    print(f"Calculated S21 phase is {S21_phase * 180 / math.pi:.2f}\N{DEGREE SIGN}.")
    print(f"Calculated S22 magnitude is {S22_mag:.2f}.")
    print(f"Calculated S22 phase is {S22_phase * 180 / math.pi:.2f}\N{DEGREE SIGN}.")
    '''

    return S11_mag, S11_phase, S12_mag, S12_phase, S21_mag, S21_phase, S22_mag, S22_phase

In [ ]:
#For one pass of a frequency list 
frequency_list = source.generate_freq_points()

# Initialize empty arrays for S-parameters
plot_S11 = np.empty(len(frequency_list), dtype=object)
plot_S12 = np.empty(len(frequency_list), dtype=object)
plot_S21 = np.empty(len(frequency_list), dtype=object)
plot_S22 = np.empty(len(frequency_list), dtype=object)

# Iterate through frequencies and compute S-parameters
for i in range(len(frequency_list)):
    source.set_frequency(frequency_list[i])  # Set the current frequency
    out_buffer = read_dma()  # Read the data buffer
    S11_mag, S11_phase, S12_mag, S12_phase, S21_mag, S21_phase, S22_mag, S22_phase = extract_S_param(out_buffer)
    
    # Store magnitude and phase for each S-parameter
    plot_S11[i] = [S11_mag, S11_phase]
    plot_S12[i] = [S12_mag, S12_phase]
    plot_S21[i] = [S21_mag, S21_phase]
    plot_S22[i] = [S22_mag, S22_phase]

In [ ]:
#Plot

# Initialize data lists
freq_data = frequency_list #Set up the frequency plot list
x_axis = [frequency_list, frequency_list, frequency_list, frequency_list]
# Initialize the plot
sparam_fig = plt.figure(title="S-Parameters Plotted", animation_duration=0)
sparam_line = plt.plot([], [], colors=["blue", "red", "green", "orange"])  # Initial empty plot

plt.xlim(0, frequency_list[len(frequency_list)-1])  # Initial X-axis range, will update dynamically
plt.xlabel("Frequency Measured [Hz]")

# Function to update the plot with new sensor data
def update_sparam_plot():
    while True:
        if sparam_is_running.value:
            #Insert function to update s-parameters if needed here
            #This would be the DMA stuff
            
            sparam_line.x = x_axis
            sparam_line.y = [plot_S11, plot_S12, plot_S21, plot_S22] #insert datapoints here 
        time.sleep(0.5) #Add delay
            
            
# Toggle button to start/stop the plot
sparam_is_running = widgets.ToggleButton(
    value=False,
    description="Running",
    icon="play",
    tooltip="Start/Stop the live plot",
)

# Display the toggle button and plot
display(sparam_is_running, sparam_fig)

# Function to start the thread for continuous plotting
def start_plot(change):
    if sparam_is_running.value:
        # Run the update function in a separate thread to avoid blocking the main thread
        sparam_thread = Thread(target=update_sparam_plot, daemon=True)
        sparam_thread.start()

# Watch the button and start the plot when pressed
is_running.observe(start_plot, names='value')

ToggleButton(value=False, description='Running', icon='play', tooltip='Start/Stop the live plot')

Figure(axes=[Axis(label='Frequency Measured [Hz]', scale=LinearScale(max=20000000000.0, min=0.0)), Axis(orient…